In [1]:
from tensorflow import keras
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0,
                                                       vertical_flip = True,
                                                       horizontal_flip = True)

In [2]:
from keras.applications import VGG16

conv_base = VGG16(weights = "imagenet",
include_top = False,
input_shape = (224, 224, 3))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [3]:
trainGen = datagen.flow_from_directory(
    r'C:\Users\vidha\Health Analytics\archive\Covid19-dataset\train',
    target_size = (224,224),
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical'
)

datatestgen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0)
                                                       
testGen = datatestgen.flow_from_directory(
    r'C:\Users\vidha\Health Analytics\archive\Covid19-dataset\test',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [4]:
import numpy as np
def extractfeatures(sample_count, generator):
    i = 0
    batch_size = 32
    features = np.zeros(shape=(sample_count, 7, 7, 512))
    labels = np.zeros(shape=(sample_count, 3))
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i+1) * batch_size] = features_batch
        labels[i * batch_size: (i+1) * batch_size] = labels_batch
        i += 1
        if ((i * batch_size % 1000) == 0 ):
            print("processed size =", i * batch_size)
        if i * batch_size >= sample_count:
            break
    return features, labels

In [5]:
train_features, train_labels = extractfeatures(251, trainGen)
test_features, test_labels = extractfeatures(66, testGen)

1/1 [==============================] - 0s 322ms/step


In [6]:
classifier = keras.Sequential([
    keras.layers.Flatten(input_shape=(7, 7, 512)),
    keras.layers.Dense(256, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation = 'softmax'),
])
classifier.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [7]:
classifier.fit(train_features, train_labels, steps_per_epoch = 251 // 32, epochs = 20, validation_split = 0.2)

Epoch 1/20
7/7 [==============================] - 3s 187ms/step - loss: 4.8279 - accuracy: 0.4250 - val_loss: 2.5056 - val_accuracy: 0.3725
Epoch 2/20
7/7 [==============================] - 1s 111ms/step - loss: 1.4766 - accuracy: 0.6100 - val_loss: 0.3374 - val_accuracy: 0.8627
Epoch 3/20
7/7 [==============================] - 1s 88ms/step - loss: 0.5079 - accuracy: 0.7700 - val_loss: 0.2953 - val_accuracy: 0.8824
Epoch 4/20
7/7 [==============================] - 1s 86ms/step - loss: 0.3498 - accuracy: 0.8400 - val_loss: 0.1915 - val_accuracy: 0.9412
Epoch 5/20
7/7 [==============================] - 1s 89ms/step - loss: 0.2271 - accuracy: 0.9150 - val_loss: 0.2125 - val_accuracy: 0.9412
Epoch 6/20
7/7 [==============================] - 1s 99ms/step - loss: 0.1785 - accuracy: 0.9400 - val_loss: 0.1584 - val_accuracy: 0.9412
Epoch 7/20
7/7 [==============================] - 1s 92ms/step - loss: 0.1226 - accuracy: 0.9700 - val_loss: 0.1444 - val_accuracy: 0.9216
Epoch 8/20
7/7 [=========

In [8]:
classifier.save('VGG16FinalProject.model')

INFO:tensorflow:Assets written to: VGG16FinalProject.model\assets


INFO:tensorflow:Assets written to: VGG16FinalProject.model\assets


In [9]:
classifier.evaluate(test_features, test_labels, batch_size = 32)

3/3 [==============================] - 0s 9ms/step - loss: 0.1190 - accuracy: 0.9242


[0.11895649880170822, 0.9242424368858337]

In [10]:
classifier.evaluate(train_features, train_labels)

8/8 [==============================] - 0s 13ms/step - loss: 0.0276 - accuracy: 0.9880


[0.027592161670327187, 0.9880478382110596]